# Database

In this file the dataset is created in which the graph data will be stored. First part is to read the *.dem* files of the matches played on inferno, than convert the datas into ***HeteroData*** structure so that it can be stored in a PyG database.

In [3]:
import torch
from torch_geometric.data import HeteroData, DataLoader
from torch_geometric.data import Dataset, Data

import matplotlib.pyplot as plt
from math import floor, ceil
import seaborn as sns
import pandas as pd
import numpy as np
import os

pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 50)

In [2]:
PATH_GRAPH_NODES = '../data/inferno_graph/graph_models/manual/nodes_v1_5.csv'
PATH_GRAPH_EDGES = '../data/inferno_graph/graph_models/manual/edges_v1_5.csv'

PATH_PARSED_DEMOS_2023 = '../data/demo-parsed/2023'
PATH_PARSED_DEMOS_2022 = '../data/demo-parsed/2022'

PATH_PLAYER_STATS = '../data/player-stats/norm_stats.csv'
PATH_PLAYER_STATS_INFERNO = '../data/player-stats/norm_stats_inf.csv'

## Creating HeteroData objects from game snapshots

Collecting the list of the inferno matches.

In [3]:
def get_inferno_matches(folder_path):
    matches = []
    for filename in os.listdir(folder_path + '/frames/'):
        if 'inferno' in filename:
            matches.append(filename)
    return matches

Reading *playerFrame*, *kills* and *rounds* dataframes.

In [4]:
def get_base_dataframes(folder_path, filename):
    playerFrames = pd.read_csv(folder_path + '/playerFrames/' + filename)
    kills = pd.read_csv(folder_path +'/kills/' + filename)
    rounds = pd.read_csv(folder_path +'/rounds/' + filename)
    bombEvents = pd.read_csv(folder_path + '/bombEvents/' + filename)
    damages = pd.read_csv(folder_path + '/damages/' + filename)


    rounds = rounds[['roundNum', 'tScore', "ctScore" ,'endTScore', 'endCTScore']]
    pf = playerFrames[['tick', 'roundNum', 'seconds', 'side', 'name', 'x', 'y', 'z','eyeX', 'eyeY', 'eyeZ', 'velocityX', 'velocityY', 'velocityZ',
        'hp', 'armor', 'activeWeapon','flashGrenades', 'smokeGrenades', 'heGrenades', 'totalUtility', 'isAlive', 'isReloading', 'isBlinded', 'isDucking',
        'isDefusing', 'isPlanting', 'isUnknown', 'isScoped', 'equipmentValue', 'equipmentValueRoundStart', 'hasHelmet','hasDefuse', 'hasBomb']]
    
    return pf, kills, rounds, bombEvents, damages

Formatting the dataframes: Calculate features, in-game live stats, etc.

In [5]:
def format_base_dataframes(pf, kills, rounds, damages):
    
    # Merge playerFrames with rounds
    pf = pf.merge(rounds, on='roundNum')
    
    # Calculate whether or not a player won the round
    pf["winsRounds"] = pf.apply(lambda x: 
        1 if ( (x['side'] == 'T') and (x['endTScore'] > x['tScore']) ) or ( (x['side'] == 'CT') and (x['endCTScore'] > x['ctScore']) )
        else 0, axis=1)

    # Kill stats
    pf['stat_kills'] = 0
    pf['stat_HSK'] = 0
    pf['stat_openKills'] = 0
    pf['stat_tradeKills'] = 0

    # Death stats
    pf['stat_deaths'] = 0
    pf['stat_openDeaths'] = 0

    # Assist stats
    pf['stat_assists'] = 0
    pf['stat_flashAssists'] = 0

    # Damage stats
    pf['stat_damage'] = 0
    pf['stat_weaponDamage'] = 0
    pf['stat_nadeDamage'] = 0
    
    # Setting kill-stats
    for index, row in kills.iterrows():

        # Kills
        pf.loc[(pf['tick'] >= row['tick']) & (pf['name'] == row['attackerName']), 'stat_kills'] += 1

        # HS-kills
        if row['isHeadshot']:
            pf.loc[(pf['tick'] >= row['tick']) & (pf['name'] == row['attackerName']), 'stat_HSK'] += 1

        # Opening-kills
        if row['isFirstKill']:
            pf.loc[(pf['tick'] >= row['tick']) & (pf['name'] == row['attackerName']), 'stat_openKills'] += 1

        # Trading-kills
        if row['isTrade']:
            pf.loc[(pf['tick'] >= row['tick']) & (pf['name'] == row['attackerName']), 'stat_tradeKills'] += 1

        # Deaths
        pf.loc[(pf['tick'] >= row['tick']) & (pf['name'] == row['victimName']), 'stat_deaths'] += 1

        # Opening deaths
        if row['isFirstKill']:
            pf.loc[(pf['tick'] >= row['tick']) & (pf['name'] == row['victimName']), 'stat_openDeaths'] += 1
            
        # Assists
        if pd.notna(row['assisterName']):
            pf.loc[(pf['tick'] >= row['tick']) & (pf['name'] == row['assisterName']), 'stat_assists'] += 1

        # Flash assists
        if row['victimBlinded'] and row['flashThrowerTeam'] != row['victimTeam']:
            pf.loc[(pf['tick'] >= row['tick']) & (pf['name'] == row['flashThrowerTeam']), 'stat_flashAssists'] += 1

    # Setting damage-stats
    for index, row in damages.iterrows():

        # All Damage
        if (row['isFriendlyFire'] == False):
            pf.loc[(pf['tick'] >= row['tick']) & (pf['name'] == row['attackerName']), 'stat_damage'] += row['hpDamageTaken']

        # Weapon Damage
        if (row['isFriendlyFire'] == False) and (row['weaponClass'] != "Grenade" and row['weaponClass'] != "Equipment"):
            pf.loc[(pf['tick'] >= row['tick']) & (pf['name'] == row['attackerName']), 'stat_weaponDamage'] += row['hpDamageTaken']
        
        # Nade Damage
        if (row['isFriendlyFire'] == False) and (row['weaponClass'] == "Grenade"):
            pf.loc[(pf['tick'] >= row['tick']) & (pf['name'] == row['attackerName']), 'stat_nadeDamage'] += row['hpDamageTaken']
        
    # Rounded-down seconds
    pf['sec'] = pf['seconds'].apply(lambda x: floor(x))
        
    return pf, kills, rounds

Get the dummie columns for active weapons.
<div style="color: red;">TODO: M4A4 and other weapon slots should be fixed!</div>

In [6]:
def get_dummies(pf):
    
    # Tracked weapons
    tracked_weapons = ['activeWeapon_AK-47','activeWeapon_Knife','activeWeapon_AWP','activeWeapon_M4A1','activeWeapon_Smoke Grenade','activeWeapon_M4A4','activeWeapon_Galil AR','activeWeapon_Desert Eagle','activeWeapon_Flashbang','activeWeapon_Glock-18','activeWeapon_USP-S']
    
    # Create dummie cols
    dummies = pd.get_dummies(pf['activeWeapon'], prefix="activeWeapon",drop_first=False)
    for col in dummies.columns:
        if col not in tracked_weapons:
            dummies = dummies.drop(col, axis=1)
    
    dummies = dummies*1
    pf = pf.merge(dummies, left_index = True, right_index = True, how = 'left')
    
    return pf

Create a player object to store player related data.

In [7]:
def create_players_dataframes(pf):
    
    startAsCTPlayerNames = pf[(pf['side'] == 'CT') & (pf['roundNum'] == 1)]['name'].unique()
    startAsTPlayerNames = pf[(pf['side'] == 'T') & (pf['roundNum'] == 1)]['name'].unique()
    players = {}

    # Team 1: start on CT side
    players[0] = pf[pf['name'] == startAsCTPlayerNames[0]].drop_duplicates(subset=['sec','roundNum']).copy()
    players[1] = pf[pf['name'] == startAsCTPlayerNames[1]].drop_duplicates(subset=['sec','roundNum']).copy()
    players[2] = pf[pf['name'] == startAsCTPlayerNames[2]].drop_duplicates(subset=['sec','roundNum']).copy()
    players[3] = pf[pf['name'] == startAsCTPlayerNames[3]].drop_duplicates(subset=['sec','roundNum']).copy()
    players[4] = pf[pf['name'] == startAsCTPlayerNames[4]].drop_duplicates(subset=['sec','roundNum']).copy()

    # Team 2: start on T side
    players[5] = pf[pf['name'] == startAsTPlayerNames[0]].drop_duplicates(subset=['sec','roundNum']).copy()
    players[6] = pf[pf['name'] == startAsTPlayerNames[1]].drop_duplicates(subset=['sec','roundNum']).copy()
    players[7] = pf[pf['name'] == startAsTPlayerNames[2]].drop_duplicates(subset=['sec','roundNum']).copy()
    players[8] = pf[pf['name'] == startAsTPlayerNames[3]].drop_duplicates(subset=['sec','roundNum']).copy()
    players[9] = pf[pf['name'] == startAsTPlayerNames[4]].drop_duplicates(subset=['sec','roundNum']).copy()
    
    return players


Find closest and n-closest map node neighbors of a player node. 

In [8]:
# Calculate closest map-graph node
def find_closest(row):
    nodes = pd.read_csv(PATH_GRAPH_NODES)
    distances = np.sqrt((nodes['x'] - row['x'])**2 + (nodes['y'] - row['y'])**2)
    return nodes.loc[distances.idxmin(), 'nodeId']

def find_n_closest(row, n=3):
    nodes = pd.read_csv(PATH_GRAPH_NODES)
    distances = np.sqrt((nodes['x'] - row['x'])**2 + (nodes['y'] - row['y'])**2)
    closest_indices = distances.argsort()[:n]
    return nodes.loc[closest_indices, 'nodeId'].tolist()

def calculate_closest_map_node_to_player(pf):
    
    # Read nodes and create players
    players = create_players_dataframes(pf)

    for idx, player in enumerate(players):
        
        players[idx] = players[idx].replace(True, 1)
        players[idx] = players[idx].replace(False, 0)
        players[idx]['isCT'] = players[idx]['side'].apply(lambda x: 1 if x == 'CT' else 0)
        players[idx]['closestId'] = players[idx].apply(find_closest, axis=1)

        del players[idx]['side']            # Deleted because isCT flag holds the info
        del players[idx]['activeWeapon']    # Deleted because of the activeWeapon_*weapon* dummie columns
        del players[idx]['winsRounds']      # Round winning team is a graph level detail
    
    return players

Get the player edges.

In [9]:
def get_player_edges(players):
    
    playerEdges = None

    for idx in range(0,len(players)):
        temp = players[idx][['roundNum','sec','closestId']].copy()
        temp['playerId'] = idx
        if playerEdges is None:
            playerEdges = temp
        else:
            playerEdges = pd.concat([playerEdges, temp])
    
    return playerEdges

Calculate graph level features like time, players alive, etc.

In [10]:
def get_graph_level_data(players, rounds):
    # Copy players object
    graph_players = {}
    for idx in range(0,len(players)):
        graph_players[idx] = players[idx].copy()

    colsNotToRename = ['tick', 'roundNum', 'seconds', 'sec']

    # Rename columns except for tick, roundNum, seconds, floorSec
    for idx in range(0,len(graph_players)):
        
        for col in graph_players[idx].columns:
            if col not in colsNotToRename:
                graph_players[idx].rename(columns={col: "player" + str(idx) + "_" + col}, inplace=True)

    # Create a graph dataframe to store all players in 1 row per second
    graph_data = graph_players[0].copy()

    # Merge dataframes
    for i in range(1, len(graph_players)):
        graph_data = graph_data.merge(graph_players[i], on=colsNotToRename)
        
    graph_data = graph_data.merge(rounds, on=['roundNum'])
    graph_data['CTwinsRound'] = graph_data.apply(lambda x: 1 if (x['endCTScore'] > x['ctScore']) else 0, axis=1)
    graph_data['team1AliveNum'] = graph_data[['player0_isAlive','player1_isAlive','player2_isAlive','player3_isAlive','player4_isAlive']].sum(axis=1)
    graph_data['team2AliveNum'] = graph_data[['player5_isAlive','player6_isAlive','player7_isAlive','player8_isAlive','player9_isAlive']].sum(axis=1)
    graph_data = graph_data[['roundNum','seconds','sec','team1AliveNum','team2AliveNum','CTwinsRound']]
    
    return graph_data

Read inferno graph model.

In [11]:
def get_inferno_graph_nodes_and_edges():
    nodes = pd.read_csv(PATH_GRAPH_NODES)
    edges = pd.read_csv(PATH_GRAPH_EDGES)
    
    nodes['x'] = nodes['x'].astype('float32')
    nodes['y'] = nodes['y'].astype('float32')
    nodes['isPath'] = nodes['type'].apply(lambda x: 1 if x == 'path' else 0)
    
    return nodes, edges

Get box-score statistics for the players.

In [12]:
def get_player_overall_statistics(players):
    # Needed columns
    needed_stats = ['player_name', 'rating_2.0', 'DPR', 'KAST', 'Impact', 'ADR', 'KPR','total_kills', 'HS%', 'total_deaths', 'KD_ratio', 'dmgPR',
       'grenade_dmgPR', 'maps_played', 'saved_by_teammatePR', 'saved_teammatesPR','opening_kill_rating', 'team_W%_after_opening',
       'opening_kill_in_W_rounds', 'rating_1.0_all_Career', 'clutches_won_1on1', 'clutches_won_1on2', 'clutches_won_1on3', 'clutches_won_1on4', 'clutches_won_1on5']
    
    stats = pd.read_csv(PATH_PLAYER_STATS)
    stats['player_name'] = stats['player_name'].apply(lambda x: x.replace('-', '') if '-' in x else x)
    stats['player_name'] = stats['player_name'].apply(lambda x: x.replace(' ', '') if '-' in x else x)
    stats = stats[needed_stats]
    stats['rating_1.0_all_Career'] = stats['rating_1.0_all_Career'].astype('float32')
    
    inf_stats = pd.read_csv(PATH_PLAYER_STATS_INFERNO)
    inf_stats['player_name'] = inf_stats['player_name'].apply(lambda x: x.replace('-', '') if '-' in x else x)
    inf_stats['player_name'] = inf_stats['player_name'].apply(lambda x: x.replace(' ', '') if '-' in x else x)
    inf_stats = inf_stats[needed_stats]
    inf_stats['rating_1.0_all_Career'] = inf_stats['rating_1.0_all_Career'].astype('float32')
    

    for col in stats.columns:
        if col != 'player_name':
            stats[col] = stats[col].astype('float32')
            stats.rename(columns={col: "overall_" + col}, inplace=True)
            
            inf_stats[col] = inf_stats[col].astype('float32')
            inf_stats.rename(columns={col: "inf_" + col}, inplace=True)
    
    for idx in range(0,len(players)):
        players[idx] = pd.merge(players[idx], stats, left_on='name', right_on='player_name', how='left').drop(columns=['player_name'])
        players[idx] = pd.merge(players[idx], inf_stats, left_on='name', right_on='player_name', how='left').drop(columns=['player_name'])
        
    return players

Format player data for HeteroData object creation.

In [13]:
def create_playerFrameData_for_graph(players, roundNum, sec):
    playerFrameData = pd.DataFrame(columns=players[0].columns)
    for idx in range(0,len(players)):
        playerFrameData.loc[idx] = players[idx].loc[(players[idx]['roundNum'] == roundNum) & (players[idx]['sec'] == sec)].iloc[0]
        
    playerFrameData.drop(columns=['tick', 'roundNum', 'seconds', 'name', 'closestId'], inplace=True)
    playerFrameData = playerFrameData.astype('float32')
    
    return playerFrameData

Create the HeteroData object.

In [14]:
def create_HeteroData(playerFrameData, playerEdges, graph_data, nodes, edges, roundNum, sec):
    data = HeteroData()

    # Create a sample heterogeneous graph with node, edge, and multiple graph-level features
    data['player'].x = torch.tensor(playerFrameData.astype('float32').values)
    data['map'].x = torch.tensor(nodes[['x','y','isPath']].astype('float32').values)

    data['map', 'connected_to', 'map'].edge_index = torch.tensor(edges.T.values)
    data['player', 'closest_to', 'map'].edge_index = torch.tensor(playerEdges.loc[ (playerEdges['roundNum'] == roundNum) & (playerEdges['sec'] == sec)][['playerId','closestId']].T.values)


    # Define multiple graph-level features
    data.y = {
        'roundNum': roundNum.astype('float32'),
        'sec': sec.astype('float32'),
        'team1AliveNum': torch.tensor(graph_data.loc[ (graph_data['roundNum'] == roundNum) & (graph_data['sec'] == sec)]['team1AliveNum'].iloc[0].astype('float32')),
        'team2AliveNum': torch.tensor(graph_data.loc[ (graph_data['roundNum'] == roundNum) & (graph_data['sec'] == sec)]['team2AliveNum'].iloc[0].astype('float32')),
        'CTwinsRound': torch.tensor(graph_data.loc[ (graph_data['roundNum'] == roundNum) & (graph_data['sec'] == sec)]['CTwinsRound'].iloc[0])
    }
    
    return data

----------
## Dataset for HeteroData objects

In [15]:
class InfernoDataset(Dataset):
    def __init__(self, data_list):
        super(InfernoDataset, self).__init__()
        self.data_list = data_list

    def len(self):
        return len(self.data_list)

    def get(self, idx):
        return self.data_list[idx]


#### Codes testing the created dataset class

In [15]:
# infdataset = InfernoDataset([data])
# torch.save(infdataset, './data/_/test_dataset.pt')
# dataset = torch.load('./data/_/test_dataset.pt')
# loader = DataLoader(infdataset, batch_size=1, shuffle=True)

---------
## Creating the dataset

In [15]:
# Consts
FOLDER_READ_PATH = '../data/demo-parsed/2023'
FOLDER_SAVE_PATH = './graph-dataset/inferno_graph_dataset.pt'
LAST_SAVED_TXT_PATH = './graph-dataset/last-saved.txt'

# Params for workflow
hetero_data_list = []
reached_last_save = False
last_saved = ''

# Get matches
matches = get_inferno_matches(FOLDER_READ_PATH)

# Get last saved demo
try:
    with open(LAST_SAVED_TXT_PATH, 'r') as file:
        last_saved = file.read()
except:
    last_saved = ''

# Start the creation of graphs
for idx,match in enumerate(matches):

    # Skip the matches with damaged data structure
    if match == 'blast-premier-spring-final-2023-g2-vs-cloud9-bo3-l6_WROLL2MODdQ0NexGWUJg2-vs-cloud9-m2-inferno.dem.csv'  or \
       match == 'iem-cologne-2023-g2-vs-faze-bo3-2rs0vwSX-MC7GnSL4ITW06g2-vs-faze-m1-inferno.dem.csv' or \
       match == 'iem-cologne-2023-g2-vs-astralis-bo3-_w4eg3Jhwy1dhctFRgGSl7g2-vs-astralis-m2-inferno.dem.csv' or \
       match == 'iem-katowice-2023-faze-vs-ihc-bo3-xzdrX8BWDhgJKqFJblorpafaze-vs-ihc-m1-inferno.dem.csv':
        continue
    
    # Skip already processed demos
    if last_saved == '':
        reached_last_save = True
    elif last_saved != match and reached_last_save == False:
        continue
    elif last_saved == match and reached_last_save == False:
        reached_last_save = True
        continue
    
    # Create graph
    pf, kills, rounds, bombEvents, damages = get_base_dataframes(FOLDER_READ_PATH, match)
    pf, kills, rounds = format_base_dataframes(pf, kills, rounds, damages)
    pf = get_dummies(pf)
    players = calculate_closest_map_node_to_player(pf)
    players = get_player_overall_statistics(players)
    player_edges = get_player_edges(players)
    graph_data = get_graph_level_data(players, rounds)
    nodes, edges = get_inferno_graph_nodes_and_edges()

    rounds_list = pf['roundNum'].unique()
    for roundNum in rounds_list:
        sec_list = pf[pf['roundNum'] == roundNum]['sec'].unique()
        for sec in sec_list:
            playerFrameData = create_playerFrameData_for_graph(players, roundNum, sec)
            data = create_HeteroData(playerFrameData, player_edges, graph_data, nodes, edges, roundNum, sec)
            hetero_data_list.append(data)
        print(match,roundNum)
    
    if idx % 5 == 0:
        if os.path.exists('./data/inferno_graph_dataset_3.pt') == False:
            infdataset = InfernoDataset(hetero_data_list)
            torch.save(infdataset, './data/inferno_graph_dataset_3.pt')
            hetero_data_list = []
        elif os.path.exists('./data/inferno_graph_dataset_3.pt') == True:
            infdataset = torch.load('./data/inferno_graph_dataset_3.pt')
            infdataset.data_list = infdataset.data_list + hetero_data_list
            torch.save(infdataset, './data/inferno_graph_dataset_3.pt')
            hetero_data_list = []
        
        with open('last-saved.txt', 'w') as file:
            file.write(match)

iem-dallas-2023-mouz-vs-og-bo3-tGd7zF8hss-uT_IOarRz-8mouz-vs-og-m2-inferno.dem.csv 1
iem-dallas-2023-mouz-vs-og-bo3-tGd7zF8hss-uT_IOarRz-8mouz-vs-og-m2-inferno.dem.csv 2
iem-dallas-2023-mouz-vs-og-bo3-tGd7zF8hss-uT_IOarRz-8mouz-vs-og-m2-inferno.dem.csv 3
iem-dallas-2023-mouz-vs-og-bo3-tGd7zF8hss-uT_IOarRz-8mouz-vs-og-m2-inferno.dem.csv 4
iem-dallas-2023-mouz-vs-og-bo3-tGd7zF8hss-uT_IOarRz-8mouz-vs-og-m2-inferno.dem.csv 5
iem-dallas-2023-mouz-vs-og-bo3-tGd7zF8hss-uT_IOarRz-8mouz-vs-og-m2-inferno.dem.csv 6
iem-dallas-2023-mouz-vs-og-bo3-tGd7zF8hss-uT_IOarRz-8mouz-vs-og-m2-inferno.dem.csv 7
iem-dallas-2023-mouz-vs-og-bo3-tGd7zF8hss-uT_IOarRz-8mouz-vs-og-m2-inferno.dem.csv 8
iem-dallas-2023-mouz-vs-og-bo3-tGd7zF8hss-uT_IOarRz-8mouz-vs-og-m2-inferno.dem.csv 9
iem-dallas-2023-mouz-vs-og-bo3-tGd7zF8hss-uT_IOarRz-8mouz-vs-og-m2-inferno.dem.csv 10
iem-dallas-2023-mouz-vs-og-bo3-tGd7zF8hss-uT_IOarRz-8mouz-vs-og-m2-inferno.dem.csv 11
iem-dallas-2023-mouz-vs-og-bo3-tGd7zF8hss-uT_IOarRz-8mouz-vs-og

----------
# Playground

### 1. Workflow test

In [16]:
FOLDER_PATH = '../data/demo-parsed/2023'
match = 'iem-dallas-2023-mouz-vs-og-bo3-tGd7zF8hss-uT_IOarRz-8mouz-vs-og-m2-inferno.dem.csv'

hetero_data_list = []

pf, kills, rounds, bombEvents, damages = get_base_dataframes(FOLDER_PATH, match)
pf, kills, rounds = format_base_dataframes(pf, kills, rounds, damages)
pf = get_dummies(pf)
players = calculate_closest_map_node_to_player(pf)
players = get_player_overall_statistics(players)
player_edges = get_player_edges(players)
graph_data = get_graph_level_data(players, rounds)
nodes, edges = get_inferno_graph_nodes_and_edges()
rounds_list = pf['roundNum'].unique()
for roundNum in rounds_list:
    sec_list = pf[pf['roundNum'] == roundNum]['sec'].unique()
    for sec in sec_list:
        playerFrameData = create_playerFrameData_for_graph(players, roundNum, sec)
        break
    break
playerFrameData.head(5)

,x,y,z,eyeX,eyeY,eyeZ,velocityX,velocityY,velocityZ,hp,armor,flashGrenades,smokeGrenades,heGrenades,totalUtility,isAlive,isReloading,isBlinded,isDucking,isDefusing,isPlanting,isUnknown,isScoped,equipmentValue,equipmentValueRoundStart,hasHelmet,hasDefuse,hasBomb,tScore,ctScore,endTScore,endCTScore,stat_kills,stat_HSK,stat_openKills,stat_tradeKills,stat_deaths,stat_openDeaths,stat_assists,stat_flashAssists,stat_damage,stat_weaponDamage,stat_nadeDamage,sec,activeWeapon_AK-47,activeWeapon_AWP,activeWeapon_Desert Eagle,activeWeapon_Flashbang,activeWeapon_Galil AR,activeWeapon_Glock-18,activeWeapon_Knife,activeWeapon_M4A1,activeWeapon_M4A4,activeWeapon_Smoke Grenade,activeWeapon_USP-S,isCT,overall_rating_2.0,overall_DPR,overall_KAST,overall_Impact,overall_ADR,overall_KPR,overall_total_kills,overall_HS%,overall_total_deaths,overall_KD_ratio,overall_dmgPR,overall_grenade_dmgPR,overall_maps_played,overall_saved_by_teammatePR,overall_saved_teammatesPR,overall_opening_kill_rating,overall_team_W%_after_opening,overall_opening_kill_in_W_rounds,overall_rating_1.0_all_Career,overall_clutches_won_1on1,overall_clutches_won_1on2,overall_clutches_won_1on3,overall_clutches_won_1on4,overall_clutches_won_1on5,inf_rating_2.0,inf_DPR,inf_KAST,inf_Impact,inf_ADR,inf_KPR,inf_total_kills,inf_HS%,inf_total_deaths,inf_KD_ratio,inf_dmgPR,inf_grenade_dmgPR,inf_maps_played,inf_saved_by_teammatePR,inf_saved_teammatesPR,inf_opening_kill_rating,inf_team_W%_after_opening,inf_opening_kill_in_W_rounds,inf_rating_1.0_all_Career,inf_clutches_won_1on1,inf_clutches_won_1on2,inf_clutches_won_1on3,inf_clutches_won_1on4,inf_clutches_won_1on5
0,2453.319092,2014.020264,128.03125,2453.319092,2014.020264,192.093811,73.908775,67.202278,0.0,100.0,100.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,850.0,200.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.750000,0.636364,0.627119,0.808081,0.888889,0.705882,0.167079,0.551648,0.127060,0.571429,0.888889,0.922078,0.103252,0.6,0.555556,0.615385,0.444444,0.610169,0.60,0.076372,0.157025,0.131579,0.055556,0.5,0.722222,0.545455,0.941423,0.664921,0.940187,0.645161,0.177125,0.497778,0.130834,0.590909,0.940187,1.000000,0.132353,0.5,0.7,0.528302,0.422727,0.467391,0.565217,0.103093,0.386364,0.105263,0.0,0.0
1,2469.730225,2089.093994,132.03125,2469.730225,2089.093994,196.093811,-18.795185,-91.806030,0.0,100.0,100.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,850.0,200.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2366.843750,2067.396729,128.03125,2366.843750,2067.396729,192.093811,-2.724549,-99.460327,0.0,100.0,100.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,850.0,200.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2289.612061,2022.598877,128.03125,2289.612061,2022.598877,192.093811,-43.289551,-90.025925,0.0,100.0,100.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,850.0,200.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.629630,0.636364,0.903766,0.643979,0.841121,0.612903,0.056102,0.624444,0.027679,0.484848,0.841121,0.691176,0.005882,0.6,0.7,0.509434,0.645455,0.570652,0.521739,0.134021,0.090909,0.105263,0.0,0.0
4,2430.139404,2152.960938,128.03125,2430.139404,2152.960938,192.093811,-61.569103,-78.663200,0.0,100.0,100.0,0.0,0.0,0.0,0.0,1.0,0.0

In [ ]:
players = get_player_overall_statistics(players)
player_edges = get_player_edges(players)
graph_data = get_graph_level_data(players, rounds)
nodes, edges = get_inferno_graph_nodes_and_edges()
rounds_list = pf['roundNum'].unique()
for roundNum in rounds_list:
    sec_list = pf[pf['roundNum'] == roundNum]['sec'].unique()
    for sec in sec_list:
        playerFrameData = create_playerFrameData_for_graph(players, roundNum, sec)
        data = create_HeteroData(playerFrameData, player_edges, graph_data, nodes, edges, roundNum, sec)
        hetero_data_list.append(data)
    print(match,roundNum)

### 2. Player name merge error fix

In [5]:
def get_names_stats_datasets():
    # Names dataset
    names = pd.read_csv('../data/player-stats/names.csv')
    names['name'] = names['name'].str.replace('NAF-FLY', 'NAF')
    names['name'] = names['name'].replace(' ', '')
    names['one'] = 1
    names['count'] = names.groupby(by='name')['one'].transform('sum')
    del names['one']

    # Stats dataset
    stats = pd.read_csv('../data/player-stats/scraped-in-2024/2022/player_stats_2022_utf.csv')

    return names.drop_duplicates(), stats

names, stats = get_names_stats_datasets()

In [56]:
stats.loc[stats['player_name'].str.contains('ter')]

,player_name,rating_2.0,DPR,KAST,Impact,ADR,KPR,total_kills,HS%,total_deaths,KD_ratio,dmgPR,grenade_dmgPR,maps_played,rounds_played,APR,saved_by_teammatePR,saved_teammatesPR,rounds_with_kils,KD_diff,total_opening_kills,total_opening_deaths,opening_kill_ratio,opening_kill_rating,team_W%_after_opening,opening_kill_in_W_rounds,0_kill_rounds,1_kill_rounds,2_kill_rounds,3_kill_rounds,4_kill_rounds,5_kill_rounds,rifle_kills,sniper_kills,smg_kills,pistol_kills,grenade_kills,other_kills,maps_W%,rating_2.0_1+,rating_2.0_1+_streak,rating_1.0_all_Career,rating_1.0_online_Career,rating_1.0_lan_Career,rating_1.0_major_Career,rating_1.0_data,weapon_data,clutches_won_1on1,clutches_lost_1on1,clutches_won_1on2,clutches_won_1on3,clutches_won_1on4,clutches_won_1on5
7,degster,1.19,0.58,74.6,1.13,76.3,0.76,2783,35.2,2118,1.31,76.3,2.3,136,3663,0.08,0.08,0.10,1878,665,445,235,1.89,1.17,73.5,16.9,1785,1206,478,159,31,4,955,1437,12,351,27,20,58.8,76.5,18,1.18 (833),1.19 (669),1.13 (164),1.16 (36),"{'rating_1.0_all_2018': '1.01 (5)', 'rating_1....","{'kills_awp': '1355', 'kills_ak47': '627', 'ki...",24,27,14,1,1,0
10,sterling,1.04,0.64,69.0,1.01,70.7,0.68,1981,33.7,1870,1.06,70.7,4.4,114,2906,0.09,0.08,0.09,1363,111,307,193,1.59,1.08,78.2,17.6,1543,894,346,98,24,1,707,913,24,316,21,7,42.1,50.9,5,1.16 (967),1.18 (860),1.04 (107),0.89 (4),"{'rating_1.0_all_2016': '0.68 (4)', 'rating_1....","{'kills_awp': '883', 'kills_ak47': '376', 'kil...",24,17,13,4,0,0
45,huNter-,1.12,0.64,72.0,1.09,79.1,0.73,2646,48.3,2332,1.13,79.1,6.3,135,3642,0.13,0.09,0.09,1752,314,351,303,1.16,1.01,76.1,13.6,1890,1082,480,159,28,3,1996,15,201,386,52,5,59.3,67.4,8,1.11 (1718),1.12 (1086),1.10 (632),1.07 (67),"{'rating_1.0_all_2015': '0.91 (21)', 'rating_1...","{'kills_ak47': '1062', 'kills_m4a1_silencer': ...",21,15,13,3,0,0
173,brutmonster,1.10,0.67,67.8,1.21,76.1,0.72,1242,30.8,1158,1.07,76.1,2.7,62,1720,0.10,0.09,0.09,819,84,254,192,1.32,1.22,70.1,21.7,901,499,235,69,14,2,264,756,12,201,9,2,38.7,62.9,6,1.06 (449),1.06 (426),1.08 (23),-,"{'rating_1.0_all_2022': '1.05 (62)', 'rating_1...","{'kills_awp': '696', 'kills_m4a1_silencer': '1...",8,5,4,3,0,0
182,olofmeister,0.88,0.75,68.1,0.72,71.2,0.59,121,55.4,153,0.79,71.2,4.2,8,204,0.14,0.12,0.08,93,-32,13,30,0.43,0.74,84.6,11.7,111,68,22,3,0,0,84,5,2,28,2,0,50.0,37.5,2,1.06 (1586),1.06 (744),1.05 (842),1.07 (134),"{'rating_1.0_all_2012': '0.81 (1)', 'rating_1....","{'kills_ak47': '46', 'kills_m4a1_silencer': '3...",1,0,0,0,0,0
184,dexter,0.97,0.71,67.1,1.00,70.8,0.63,2595,47.6,2919,0.89,70.8,4.9,156,4113,0.13,0.10,0.08,1780,-324,444,501,0.89,1.00,74.8,15.3,2333,1166,449,132,30,3,1972,40,128,418,40,10,59.6,39.7,4,1.05 (1231),1.11 (771),0.96 (460),0.92 (53),"{'rating_1.0_all_2015': '0.63 (4)', 'rating_1....","{'kills_ak47': '974', 'kills_m4a1_silencer': '...",19,13,6,3,0,0
266,Forester,1.10,0.66,70.2,1.12,78.0,0.70,3380,51.8,3175,1.06,78.0,3.6,174,4826,0.13,0.08,0.09,2287,205,546,454,1.20,1.08,74.0,16.2,2539,1477,576,191,37,6,2805,7,72,456,40,14,52.9,66.7,13,1.03 (1513),1.03 (1372),1.02 (141),0.94 (33),"{'rating_1.0_all_2017': '0.99 (32)', 'rating_1...","{'kills_ak47': '1435', 'kills_m4a1_silencer': ...",47,14,19,8,0,0
271,buster,0.95,0.61,69.7,0.82,65.0,0.57,724,43.5,780,0.93,65.0,5.3,48,1279,0.12,0.07,0.08,518,-56,94,121,0.78,0.88,76.6,10.9,761,361,120,25,12,0,596,27,2,91,11,4,54.2,39.6,5,1.02 (1384),1.04 (996),0.98 (388),1.00 (71),"{'rating_1.0_all_2016': '0.96 (25)', 'rating_1...","{'kills_ak47': '311', 'kills_m4a1_silencer': '...",9,7,3,0,0,0
278,b0denmaster,1.14,0.61,71.2,1.17,74.6,0.71,4795,29.0,4110,1.17,74.6,4.0,258,6736,0.10,0.08,0.08,3220,685,886,570,1.55,1.18,75.7,18.5,3516,2024,879,260,52,5,1209,2735,171,621,39,36,61.2,68.6,9,1.02 (956),1.02 (839),1.07 (117),-,"{'rating_1.0_all_2017': '1.13 (3)', 'rating_1....","{'kills_awp': '2648', 'kills_ak47': '649', 'ki...",53,32,37,8,2,0
364,waterfaLLZ,1.03,0.60,67.8,0.99,66.8,0.64,759,24.5,710,1.07,66.8,2.8,42,1188,0.08,0.07,0.07,522,49,125,94,1.33,1.06,76.0,15.6,666,338,142,33,

>Data needs to be collected manually for these players:

  - huNter ✅
  - HooXi ✅
  - rain ✅
  - broky ✅
  - oSee ✅
  - NAF ✅
  - jabbi ✅
  - b1t
  - dev1ce
  - Grim
  - Hobbit
  - NiKo
  - electronic
  - SH1R0
  - xertioN
  - niko

In [54]:
mg = pd.merge(names, stats, left_on='name', right_on='player_name', how='left')
mg.loc[mg["player_name"].isna()].tail(50)

,name,count,player_name,rating_2.0,DPR,KAST,Impact,ADR,KPR,total_kills,HS%,total_deaths,KD_ratio,dmgPR,grenade_dmgPR,maps_played,rounds_played,APR,saved_by_teammatePR,saved_teammatesPR,rounds_with_kils,KD_diff,total_opening_kills,total_opening_deaths,opening_kill_ratio,opening_kill_rating,team_W%_after_opening,opening_kill_in_W_rounds,0_kill_rounds,1_kill_rounds,2_kill_rounds,3_kill_rounds,4_kill_rounds,5_kill_rounds,rifle_kills,sniper_kills,smg_kills,pistol_kills,grenade_kills,other_kills,maps_W%,rating_2.0_1+,rating_2.0_1+_streak,rating_1.0_all_Career,rating_1.0_online_Career,rating_1.0_lan_Career,rating_1.0_major_Career,rating_1.0_data,weapon_data,clutches_won_1on1,clutches_lost_1on1,clutches_won_1on2,clutches_won_1on3,clutches_won_1on4,clutches_won_1on5
35,jabbi,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,b1t,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,dev1ce,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,Grim,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,JT,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,floppy,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,nafany,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,Hobbit,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,wiz,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,neaLaN,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 3. Other tests

In [52]:
import torch

pfd_pd = playerFrameData.drop(columns=['activeWeapon_M4A4']).copy()
pfd = torch.tensor(pfd_pd.astype('float32').values)
pfd = torch.cat((pfd[:, : -3], 69*torch.ones((10,1)), pfd[:, -3:]), dim=1)

In [53]:
pd.DataFrame(pfd.numpy(), columns=playerFrameData.columns)

,x,y,z,eyeX,eyeY,eyeZ,velocityX,velocityY,velocityZ,hp,armor,flashGrenades,smokeGrenades,heGrenades,totalUtility,isAlive,isReloading,isDefusing,isPlanting,isUnknown,equipmentValue,equipmentValueRoundStart,hasHelmet,hasDefuse,hasBomb,tScore,ctScore,endTScore,endCTScore,kills,deaths,sec,activeWeapon_AK-47,activeWeapon_AWP,activeWeapon_Desert Eagle,activeWeapon_Flashbang,activeWeapon_Galil AR,activeWeapon_Glock-18,activeWeapon_Knife,activeWeapon_M4A1,activeWeapon_M4A4,activeWeapon_Smoke Grenade,activeWeapon_USP-S,isCT
0,2492.222412,2071.601318,132.031250,2492.222412,2071.601318,196.093811,-76.375160,170.368362,0.000000,100.0,100.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,850.0,200.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,69.0,0.0,0.0,1.0
1,2406.190186,2038.682129,128.031250,2406.190186,2038.682129,192.093811,-94.908333,-182.908722,0.000000,100.0,100.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,850.0,200.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,69.0,0.0,0.0,1.0
2,2352.146484,1842.617920,128.031250,2352.146484,1842.617920,192.093811,-44.704262,-242.755402,0.000000,100.0,100.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,850.0,200.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,69.0,0.0,0.0,1.0
3,2156.247559,1846.434814,171.846405,2156.247559,1846.434814,235.908966,-277.164856,-32.008446,-138.631622,100.0,100.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,850.0,200.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,69.0,0.0,0.0,1.0
4,2292.158936,1971.556152,128.031250,2292.158936,1971.556152,192.093811,-110.937187,-214.246552,0.000000,100.0,100.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,850.0,200.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,69.0,0.0,0.0,1.0
5,-1524.365234,464.814362,-64.873459,-1524.365234,464.814362,-0.810898,92.401840,-232.297012,0.000000,100.0,100.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,850.0,200.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,69.0,0.0,0.0,0.0
6,-1493.329956,273.753601,-61.961029,-1493.329956,273.753601,2.101532,206.466843,-128.398819,0.000000,100.0,100.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,850.0,200.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,69.0,0.0,0.0,0.0
7,-1482.193848,349.312958,-64.742050,-1482.193848,349.312958,-0.679489,73.808731,-225.912552,0.000000,100.0,0.0,1.0,1.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,1100.0,200.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,69.0,0.0,0.0,0.0
8,-1432.606445,240.619583,-60.628437,-1432.606445,240.619583,3.434124,244.682693,-51.286919,0.000000,100.0,100.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,850.0,200.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,69.0,0.0,0.0,0.0
9,-1516.122681,360.696625,-66.276566,-1516.122681,360.696625,-2.214005,13.836136,-190.604813,0.000000,100.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,800.0,200.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,69.0,0.0,0.0,0.0
